## MCP (Model Context Protocol) Tool Integration

This notebook demonstrates how to integrate MCP servers with LangChain agents using OCI Generative AI. It shows both manual tool calling and automatic agent execution with external MCP tools.

## What this notebook does

Demonstrates MCP server integration with LangChain, showing how to connect to weather and bill projection MCP servers, manually call tools, and use automatic agent execution.

## Documentation to reference

- OCI Gen AI: https://docs.oracle.com/en-us/iaas/Content/generative-ai/pretrained-models.htm
- LangChain: https://docs.langchain.com/oss/python/langchain/agents
- MCP Servers: https://modelcontextprotocol.io/docs/learn/server-concepts
- LangChain MCP clients: https://docs.langchain.com/oss/python/langchain/mcp#model-context-protocol-mcp
- OCI OpenAI compatible SDK: https://github.com/oracle-samples/oci-openai

## Relevant slack channels

- #generative-ai-users: for questions on OCI Gen AI
- #igiu-innovation-lab: general discussions on your project
- #igiu-ai-learning: help with sandbox environment or help with running this code

## Env setup

- sandbox.yaml: Contains OCI config, compartment, DB details, and wallet path.
- .env: Load environment variables (e.g., API keys if needed).
- configure cwd for jupyter match your workspace python code: 
    -  vscode menu -> Settings > Extensions > Jupyter > Notebook File Root
    -  change from `${fileDirname}` to `${workspaceFolder}`
- MCP servers must be running:
  - Weather server on http://localhost:8000/mcp
  - Bill server via stdio

## How to run the notebook

1. Start the MCP servers first:
   - `uv run langChain/function_calling/mcp/weather_mcp_server.py` (in background)
   - The bill server will be started automatically by the MCP client
2. Run the cells in order. Make sure your sandbox.yaml is configured.

## Comments to important sections of notebook

- Cell 1: Instantiate the OCI client and load config
- Cell 2: Set up MCP client connections
- Manual section: Demonstrates manual MCP tool calling

## Experimentation Tips

- Try different queries that combine weather and bill information
- Experiment with the system_prompt in the agent
- Test error handling when MCP servers are unavailable

In [ ]:
import sys, os
from dotenv import load_dotenv
from envyaml import EnvYAML
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.messages import HumanMessage
from langChain.oci_openai_helper import OCIOpenAIHelper

def load_config(config_path):
    try:
        with open(config_path, 'r') as f:
            return EnvYAML(config_path)
    except FileNotFoundError:
        print(f"Error: Configuration file '{config_path}' not found.")
        return None

load_dotenv()
SANDBOX_CONFIG_FILE = "sandbox.yaml"
LLM_MODEL = "openai.gpt-4.1"

# Step 1: Load configuration and initialize OCI client
scfg = load_config(SANDBOX_CONFIG_FILE)

llm_client = OCIOpenAIHelper.get_client(
    model_name=LLM_MODEL,
    config=scfg
)

print("OCI client initialized successfully!")

## 2. Set up MCP client connections

Initialize the MCP client to connect to multiple servers (weather and bill projection). This cell demonstrates how to establish connections to external MCP servers.

In [ ]:
# Step 2: Set up MCP client with multiple servers
async def setup_mcp_client():
    """
    Initialize MCP client with connections to weather and bill servers.
    
    Make sure the weather server is running on localhost:8000
    The bill server will be started automatically via stdio
    """
    client = MultiServerMCPClient(
        {
            "weather": {
                "transport": "streamable_http",
                "url": "http://localhost:8000/mcp",
            },
            "bill_server": {
                "command": "python",
                "args": ["./langChain/function_calling/mcp/bill_mcp_server.py"],
                "transport": "stdio",
            },
        }
    )
    
    tools = await client.get_tools()
    
    print(f"Connected to MCP servers. Found {len(tools)} tools:")
    for tool in tools:
        print(f"- {tool.name}: {tool.description}")
    
    # Bind tools to the LLM client
    tooled_model = llm_client.bind_tools(tools)
    tools_by_name = {tool.name: tool for tool in tools}
    
    return tooled_model, tools_by_name

# Test the MCP setup
import asyncio
tooled_model, tools_by_name = await setup_mcp_client()

## Manual MCP tool calling

This section demonstrates manual tool calling with MCP tools. The process is:
1. Send a message to the model
2. Check if the model wants to call tools
3. Execute the tools manually
4. Send the results back to get the final response

In [ ]:
# Manual MCP tool calling demonstration
async def manual_mcp_calling():
    prompt = "What is my projected bill if the weather is 40 degrees and I have a gas oven? Also, show me weather alerts for Colorado."
    messages = [HumanMessage(prompt)]
    
    print(f"User query: {prompt}")
    print("=" * 60)
    
    while True:
        # 1. Get AI response
        ai_message = tooled_model.invoke(messages)
        print(f"\nAI Response: {ai_message.content}")
        
        # 2. Check for tool calls
        if not getattr(ai_message, "tool_calls", None):
            print("\nNo tool calls detected — conversation finished.")
            break
        
        print(f"\n*** Tool calls detected: {len(ai_message.tool_calls)} ***")
        messages.append(ai_message)
        
        # 3. Execute all tool calls
        for tool_call in ai_message.tool_calls:
            tool_name = tool_call["name"].lower()
            selected_tool = tools_by_name.get(tool_name)
            
            if not selected_tool:
                print(f"Unknown tool requested: {tool_name}")
                continue
            
            print(f"\nExecuting tool: {tool_name}")
            tool_msg = await selected_tool.ainvoke(tool_call)
            print(f"Tool result: {tool_msg.content}")
            
            # 4. Add tool result to conversation
            messages.append(tool_msg)
    
    return messages

# Run manual MCP calling
final_messages = await manual_mcp_calling()
print("\n" + "=" * 60)
print("Manual MCP calling completed!")